# **Finding Lane Lines on the Road** 
***
In this project, you will use the tools you learned about in the lesson to identify lane lines on the road.  You can develop your pipeline on a series of individual images, and later apply the result to a video stream (really just a series of images). Check out the video clip "raw-lines-example.mp4" (also contained in this repository) to see what the output should look like after using the helper functions below. 

Once you have a result that looks roughly like "raw-lines-example.mp4", you'll need to get creative and try to average and/or extrapolate the line segments you've detected to map out the full extent of the lane lines.  You can see an example of the result you're going for in the video "P1_example.mp4".  Ultimately, you would like to draw just one line for the left side of the lane, and one for the right.

---
Let's have a look at our first image called 'test_images/solidWhiteRight.jpg'.  Run the 2 cells below (hit Shift-Enter or the "play" button above) to display the image.

**Note** If, at any point, you encounter frozen display windows or other confounding issues, you can always start again with a clean slate by going to the "Kernel" menu above and selecting "Restart & Clear Output".

---

**The tools you have are color selection, region of interest selection, grayscaling, Gaussian smoothing, Canny Edge Detection and Hough Tranform line detection.  You  are also free to explore and try other techniques that were not presented in the lesson.  Your goal is piece together a pipeline to detect the line segments in the image, then average/extrapolate them and draw them onto the image for display (as below).  Once you have a working pipeline, try it out on the video stream below.**

---

<figure>
 <img src="line-segments-example.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your output should look something like this (above) after detecting line segments using the helper functions below </p> 
 </figcaption>
</figure>
 <p></p> 
<figure>
 <img src="laneLines_thirdPass.jpg" width="380" alt="Combined Image" />
 <figcaption>
 <p></p> 
 <p style="text-align: center;"> Your goal is to connect/average/extrapolate line segments to get output like this</p> 
 </figcaption>
</figure>

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

In [ ]:
#reading in an image
image = mpimg.imread('test_images/solidWhiteRight.jpg')
#printing out some stats and plotting
print('This image is:', type(image), 'with dimesions:', image.shape)
plt.imshow(image)  #call as plt.imshow(gray, cmap='gray') to show a grayscaled image

**Some OpenCV functions (beyond those introduced in the lesson) that might be useful for this project are:**

`cv2.inRange()` for color selection  
`cv2.fillPoly()` for regions selection  
`cv2.line()` to draw lines on an image given endpoints  
`cv2.addWeighted()` to coadd / overlay two images
`cv2.cvtColor()` to grayscale or change color
`cv2.imwrite()` to output images to file  
`cv2.bitwise_and()` to apply a mask to an image

**Check out the OpenCV documentation to learn about these and discover even more awesome functionality!**

Below are some helper functions to help get you started. They should look familiar from the lesson!

In [ ]:
import math
import operator

def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    you should call plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    color_blue=[0, 0, 255] 
    #Arrays to hold slopes and lines associated with each side of lane
    left_slopes = []
    right_slopes = []
    left_lines_x = []
    left_lines_y = []
    right_lines_x = []
    right_lines_y = []
    left_length = []
    right_length = []  

    #max left and right length of lines
    maxLengthRight = 0
    maxLengthLeft = 0
    
    #Go through each line
    #Determine if it is a right or left line
    #then interpolate the lane line based on each section of the line
    for line in lines:
        for x1,y1,x2,y2 in line:
            #calculate slope and length of line
            slope = (((y2-y1)/(x2-x1)))
            length = np.sqrt((x2-x1)*(x2-x1)+(y2-y1)*(y2-y1))
                
            #Find left line
            if (slope < -0.2):
                left_slopes.append(slope)
                left_lines_x.append(x1)
                left_lines_x.append(x2)
                left_lines_y.append(y1)
                left_lines_y.append(y2)
                left_length.append(length)
            #find right line
            if (slope > 0.2):
                right_slopes.append(slope)
                right_lines_x.append(x1)
                right_lines_x.append(x2)
                right_lines_y.append(y1)
                right_lines_y.append(y2)
                right_length.append(length)

    #get max slope
    #index, value = max(enumerate(left_length), key=operator.itemgetter(1))
    #left_slope = left_slopes[index]#np.mean(np.array(right_slopes))
    #get average of all slopes
    left_slope = np.mean(np.array(left_slopes))               
        
    #make sure we have valid slopes
    if abs(left_slope) > 0.1 and ~np.isinf(left_slope):
        lx1 = np.average(left_lines_x)
        ly1 = np.average(left_lines_y)
        #calculate the interpolcated line and draw it
        y_intercept = ly1 -lx1 * left_slope
        left_y_min = 0.35 * image.shape[1]
        left_y_max = image.shape[1]        
        left_top = np.array([(left_y_min-y_intercept)/left_slope, left_y_min])
        left_bottom = np.array([(left_y_max-y_intercept)/left_slope, left_y_max])
        cv2.line(img, (int(left_top[0]), int(left_top[1])), (int(left_bottom[0]), int(left_bottom[1])), color, thickness*4)
    
    #get max slope
    #index, value = max(enumerate(right_length), key=operator.itemgetter(1))
    #right_slope = right_slopes[index]#np.mean(np.array(right_slopes))
    #get average of all slopes
    right_slope = np.mean(np.array(right_slopes))        
             
    if abs(right_slope) > 0.1 and ~np.isinf(right_slope):
        rx1 = np.average(right_lines_x)
        ry1 = np.average(right_lines_y)
        #calculate the interpolcated line and draw it
        y_intercept = ry1 -rx1 * right_slope
        right_y_min = 0.35 * image.shape[1]
        right_y_max = image.shape[1]
        right_top = np.array([(right_y_min-y_intercept)/right_slope, right_y_min])
        right_bottom = np.array([(right_y_max-y_intercept)/right_slope, right_y_max])
        cv2.line(img, (int(right_top[0]), int(right_top[1])), (int(right_bottom[0]), int(right_bottom[1])), color, thickness*4)
         
    #print(len(lines))
    return img
       
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((*img.shape, 3), dtype=np.uint8)
    return line_img, lines

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, a=0.8, ß=1., q=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * a + img * ß + ?
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, a, img, ß, q)

## Test on Images

Now you should build your pipeline to work on the images in the directory "test_images"  
**You should make sure your pipeline works well on these images before you try the videos.**

In [ ]:
import os, glob
os.listdir("test_images/")

for file in glob.glob("test_images/*.jpg"):
    image = mpimg.imread(file)
        
    # Convert BGR to HSV
    #color_select_image = np.copy(image)
    #hsv_image = cv2.cvtColor(color_select_image, cv2.COLOR_RGB2HSV)#, image, 0)
    
    # define range of yellow and white color in HSV
    #lower_yellow = np.array([50, 50, 50], np.uint8)
    #upper_yellow = np.array([80, 80, 80], np.uint8)    
    #lower_white = np.array([0, 0, 100], np.uint8)
    #upper_white = np.array([0, 0, 150], np.uint8)
    
    # Threshold the HSV image to get only blue colors
    #yellow_image = cv2.inRange(hsv_image, lower_yellow , upper_yellow )
    #white_image = cv2.inRange(hsv_image, lower_white , upper_white )
    #yellow_and_white_image = cv2.bitwise_or(yellow_image,white_image)
    
    # convert to grayscale
    gray_image = grayscale(image)
    equ_image = cv2.equalizeHist(gray_image);
    gaussian_image = gaussian_blur(equ_image, 3)
    ret, black_white_image = cv2.threshold(gaussian_image, 150,255, cv2.THRESH_BINARY)
  
    #preprocessed image
    #color_processed_image = cv2.bitwise_or(equ_image,yellow_and_white_image)
    canny_image = canny(black_white_image, 50, 150)
        
    ysize = image.shape[0]
    xsize = image.shape[1]

    left_bottom = [0, ysize]
    right_bottom = [xsize, ysize]
    apex = [xsize/2, ysize/2]
    vertices = np.array([[(0,ysize),(xsize*.5,ysize*.6), (xsize*.5,ysize*.6), (xsize,ysize)]], dtype=np.int32)

    roi_image = region_of_interest(canny_image, vertices)
    rho = 5 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 25     # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 50 #minimum number of pixels making up a line
    max_line_gap = 50    # maximum gap in pixels between connectable line segments

    hough_image, hough_line_list = hough_lines(roi_image, rho, theta, threshold, min_line_len, max_line_gap)
    line_image = draw_lines(hough_image, hough_line_list);
    result = weighted_img(line_image, image)
    filename = 'result_' + file
    print(filename)
    cv2.imwrite(filename, result)
    plt.imshow(result)
    


run your solution on all test_images and make copies into the test_images directory).

## Test on Videos

You know what's cooler than drawing lanes over images? Drawing lanes over video!

We can test our solution on two provided videos:

`solidWhiteRight.mp4`

`solidYellowLeft.mp4`

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML





In [ ]:
def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image with lines are drawn on lanes)
    left_slope = 0
    right_slope = 0
    color_select_image = np.copy(image)
    
    # Convert BGR to HSV
    hsv_image = cv2.cvtColor(color_select_image, cv2.COLOR_RGB2HSV)#, image, 0)
    
    # define range of yellow and white color in HSV
    #lower_yellow = np.array([50, 50, 50], np.uint8)
    #upper_yellow = np.array([80, 80, 80], np.uint8)    
    #lower_white = np.array([0, 0, 100], np.uint8)
    #upper_white = np.array([0, 0, 150], np.uint8)
    
    # Threshold the HSV image to get only blue colors
    #yellow_image = cv2.inRange(hsv_image, lower_yellow , upper_yellow )
    #white_image = cv2.inRange(hsv_image, lower_white , upper_white )
    #yellow_and_white_image = cv2.bitwise_or(yellow_image,white_image)
    
    # convert to grayscale
    gray_image = grayscale(image)
    equ_image = cv2.equalizeHist(gray_image);
    gaussian_image = gaussian_blur(equ_image, 3)
    ret, black_white_image = cv2.threshold(gaussian_image, 150,255, cv2.THRESH_BINARY)
    #plt.imshow(black_white_image, cmap="Greys_r")
        
    #preprocessed image
    #color_processed_image = cv2.bitwise_or(equ_image,yellow_and_white_image)
 
    canny_image = canny(black_white_image, 50, 150)
    
    ysize = image.shape[0]
    xsize = image.shape[1]

    left_bottom = [0, ysize]
    right_bottom = [xsize, ysize]
    apex = [xsize/2, ysize/2]
    vertices = np.array([[(0,ysize),(xsize*.5,ysize*.55), (xsize*.5,ysize*.55), (xsize,ysize)]], dtype=np.int32)

    roi_image = region_of_interest(canny_image, vertices)
    rho = 5 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 60     # minimum number of votes (intersections in Hough grid cell)
    min_line_len = 100 #minimum number of pixels making up a line
    max_line_gap = 60    # maximum gap in pixels between connectable line segments

    hough_image, hough_line_list = hough_lines(roi_image, rho, theta, threshold, min_line_len, max_line_gap)
    line_image = draw_lines(hough_image, hough_line_list);
    result = weighted_img(line_image, image)
    #backtorgb = cv2.cvtColor(roi_image,cv2.COLOR_GRAY2BGR)
    #result = weighted_img(line_image, backtorgb)
    #plt.imshow(line_image)

    return result

Let's try the one with the solid white lane on the right first ...

In [ ]:
white_output = 'white.mp4'
clip1 = VideoFileClip("solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

Play the video inline, or if you prefer find the video in your filesystem (should be in the same directory) and play it in your video player of choice.

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

**At this point, if you were successful you probably have the Hough line segments drawn onto the road, but what about identifying the full extent of the lane and marking it clearly as in the example video (P1_example.mp4)?  Think about defining a line to run the full length of the visible lane based on the line segments you identified with the Hough Transform.  Modify your draw_lines function accordingly and try re-running your pipeline.**

Now for the one with the solid yellow lane on the left. This one's more tricky!

In [ ]:
yellow_output = 'yellow.mp4'
clip2 = VideoFileClip('solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

## Reflections

Congratulations on finding the lane lines!  As the final step in this project, we would like you to share your thoughts on your lane finding pipeline... specifically, how could you imagine making your algorithm better / more robust?  Where will your current algorithm be likely to fail?

Please add your thoughts below,  and if you're up for making your pipeline more robust, be sure to scroll down and check out the optional challenge video below!

Denise's comments:

In order to improve the algorithm, the following could be done:

-Parameters should be set based on a learning algorithm.  Right now, all parameters were set empirically are only work with a small set of data

-Parameters could be specific to the environment.  Instead of parameters that cover all criteria, select the parameters based on the environemt.  That may mean there are different parameters for different lighting conditions (day, night, overcast) or for highway versus city streets.

-When identifying lines (or other items in the scene) take into account the position of those items in the previous frame. Since we know what direction the car is moving, we can estimate where the objects should be in the next frame.

-Instead of assuming lane lines are straight lines, allow them to be curved and look for contours instead of straight lines.  


## Submission

If you're satisfied with your video outputs it's time to submit!  Submit this ipython notebook for review.


## Optional Challenge

Try your lane finding pipeline on the video below.  Does it still work?  Can you figure out a way to make it more robust?  If you're up for the challenge, modify your pipeline so it works with this video and submit it along with the rest of your project!

In [ ]:
challenge_output = 'extra.mp4'
clip2 = VideoFileClip('challenge.mp4')
challenge_clip = clip2.fl_image(process_image)
%time challenge_clip.write_videofile(challenge_output, audio=False)

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(challenge_output))